In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from cellpose import models
from tifffile import imread
from skimage.io import imsave


In [ ]:
# load cellpose model (set gpu to True if gonna use gpu)
model = models.Cellpose(gpu=False, model_type='cyto')

In [ ]:
im_dir = r'E:\labwork\test\data'
save_dir = r'E:\labwork\test\segmentation'
exp_dir = r'E:\labwork\test'

In [ ]:
folder_path=im_dir
# select all TIFF/TIF files in the folder
im_file = [filename for filename in os.listdir(im_dir) if filename.endswith('.tif')]  # change to '.tiff' if needed

for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)

        # get an image
    im = imread(filepath)

        # segment the right plane
    labels, _, _, _ = model.eval(im, diameter=30, channels=[0,0])
    print(f'segmented')
        # save segmentation
    path_save_temp = os.path.join(save_dir,filename)
    imsave(path_save_temp,labels)
    
#     # or you could use this to save
#     save_file = file_name[:-4]+f'_{str(i).zfill(3)}_label.png' # or change to another format of your choice
#     save_path = os.path.join(save_dir,save_file)
#     imsave(save_path,labels.astype('uint16'))

In [ ]:
labels_dir = os.path.join(exp_dir,'segmentation')
im_dir = os.path.join(exp_dir,'data')
df_dir = os.path.join(exp_dir,'qf')

In [ ]:
# reading labels 
file_list = [x for x in os.listdir(labels_dir) if 'd17' in x]  # or other selection criteria of your choice
print(file_list)

In [ ]:
from skimage.io import imread
from skimage import measure
from skimage.segmentation import clear_border

In [ ]:
# setting properties to calculate, can add more
properties = ['label','area','mean_intensity']

In [ ]:
# quantification

cellDataList=[] 

for label_file in file_list:

    label_path = os.path.join(labels_dir,label_file)
    labels_2D = imread(label_path)
    labels_2D = clear_border(labels_2D)
    
    print ('read images for intensity calculations')
    intensity_list = []

    im = imread(filepath)

    intensity_list.append(im)

    int_im = np.moveaxis(np.array(intensity_list),0,2)

    print('calculate properties of regions') 
    cellData = pd.DataFrame(measure.regionprops_table(labels_2D, properties=properties,intensity_image=int_im))
    print(label_file)
    cellData['file'] = label_file

    cellDataList.append(cellData)

print ('put all together') 
cellDataAll = pd.concat(cellDataList,ignore_index=True)

# rename columns
cellDataAll.columns = ['label', 'area',
                       'mean_intensity','file'] 

# add info
cellDataAll['size_x'] = labels_2D.shape[0]
cellDataAll['size_y'] = labels_2D.shape[1]

# save calculations, 
df_name='triple_reporter_d17_etop.csv'
cellDataAll.to_csv(os.path.join(df_dir,df_name),index=False)